In [ ]:
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import time

import category_encoders as ce
import catboost
from sklearn import metrics

import sys
sys.path.append("../utils")
from metrics import compute_recall_at4, compute_normalized_gini, compute_amex_metric
#from messaging import send_message

pd.set_option('display.max_columns', None)

In [ ]:
# metrics in catboost format
class AmexMetric:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):    
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = approxes[0]
        target = target
        return compute_amex_metric(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

class RecallAt4:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = np.array(approxes[0])
        target = np.array(target)                                   
        return compute_recall_at4(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

class NormGini:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = np.array(approxes[0])
        target = np.array(target)                   
        return compute_normalized_gini(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

In [ ]:
# CONFIG PARAMS
N_REPEATS = 3

In [ ]:
OOF_PATH = Path("../data/oof/catboost-dsv06")
SUB_PATH = Path("../data/subs/catboost-dsv06")
ART_PATH = Path("../artifacts/catboost-dsv06")

if not OOF_PATH.exists():
    OOF_PATH.mkdir(parents=True, exist_ok=True)
if not SUB_PATH.exists():
    SUB_PATH.mkdir(parents=True, exist_ok=True)
if not ART_PATH.exists():
    ART_PATH.mkdir(parents=True, exist_ok=True)

***
## load and prepare data

In [ ]:
!ls ../data/processed/dsv06

In [ ]:
train = pd.read_parquet("../data/processed/dsv06/train.parquet")
train_labels = pd.read_csv("../data/raw/train_labels.csv", index_col="customer_ID")

In [ ]:
input_feats = train.columns.tolist()
len(input_feats)

In [ ]:
train = pd.merge(train, train_labels, how="inner", left_index=True, right_index=True)
del train_labels
gc.collect()

***
## model training

train with repeated cross validation

In [ ]:
model_params = {
    'eval_metric':AmexMetric(),
    'learning_rate': 0.05,
    'nan_mode':'Min',
    'random_seed': 2112,
    'auto_class_weights': None,
    'bootstrap_type': 'Bernoulli',
    'depth': 6,
    'rsm': 0.1,
    'iterations': 5000,
    'l2_leaf_reg': 37.05327214190692,
    'min_data_in_leaf': 600,
    'random_strength': 27.64341860474617,
    'subsample': 0.9,
    # early stopping
    'early_stopping_rounds':5000,
    'use_best_model': False,
}

In [ ]:
def train_models(dataframe: pd.DataFrame, n_folds: int = 5,) -> tuple:
    
    models = list()
    
    # dataframe to store the oof predictions
    oof = dataframe[["target"]].copy()
    oof["pred"] = -1

    for fold in range(n_folds):
        
        print(f" training model {fold+1}/{n_folds} ".center(100, "#"))
        
        train_df = dataframe.query("fold != @fold").copy()
        valid_df = dataframe.query("fold == @fold").copy()
                
        train_dset = catboost.Pool(
            data=train_df.loc[:,input_feats],
            label=train_df.loc[:,"target"].values,
        )
        valid_dset = catboost.Pool(
            data=valid_df.loc[:,input_feats],
            label=valid_df.loc[:,"target"].values,
        )
        
        model = catboost.CatBoostClassifier(**model_params)
        model.fit(
            train_dset,
            eval_set=valid_dset,
            verbose=25,
        )
        
        #lgb.plot_importance(model, figsize=(8,15), importance_type="split", max_num_features=30)
        #lgb.plot_importance(model, figsize=(8,15), importance_type="gain", max_num_features=30)
        #plt.show()        
        
        oof.loc[valid_df.index,"pred"] = model.predict(valid_dset, prediction_type="Probability")[:,1]
        
        models.append(model)
        del train_df,valid_df,train_dset,valid_dset
        gc.collect()
    
    return models,oof

In [ ]:
# implement repeated cross validation
sorted(glob("../data/processed/cv*.csv"))

In [ ]:
%%time 

all_models = list()
all_oof_dfs = list()

for repetition in range(N_REPEATS):
    print(f" repeated cross-validation step: {repetition+1}/{N_REPEATS} ".center(100, "#"))

    folds = pd.read_csv(f'../data/processed/cv{repetition}.csv', index_col="customer_ID")
    _train = pd.merge(train, folds, how="inner", left_index=True, right_index=True).reset_index(drop=True)
    
    tic = time.time()
    models,oof = train_models(_train)
    tac = time.time()
    print(f"Training time: {(tac-tic)/60} min.")
          
    # oof metrics
    print("OOF recall_at4:", compute_recall_at4(oof.target.values, oof.pred.values))
    print("OOF normalized_gini:", compute_normalized_gini(oof.target.values, oof.pred.values))
    print("OOF competition metric:", compute_amex_metric(oof.target.values, oof.pred.values))
    
    all_models.append(models)
    all_oof_dfs.append(oof)
    
    # save oof predictions
    oof.to_csv(OOF_PATH/f"oof-cv{repetition}.csv", index=False)
    # save models
    for fold,_model in enumerate(models):
        _model.save_model(ART_PATH/f"/model-cv{repetition}-fold{fold}.cbm", format="cbm")
          
    del _train, folds; gc.collect()

In [ ]:
results = list()

for oof in all_oof_dfs:  
    r = {
        "recall_at4": compute_recall_at4(oof.target.values, oof.pred.values),
        "gini": compute_normalized_gini(oof.target.values, oof.pred.values),
        "metric": compute_amex_metric(oof.target.values, oof.pred.values),
    }
    results.append(r)

results = pd.DataFrame(results)
display(results)

print("\nmean:")
display(results.mean(axis=0))

print("\nstd:")
display(results.std(axis=0))

In [ ]:
del train
gc.collect()

***
## make predictions and submit

In [ ]:
def make_predictions(dataframe:pd.DataFrame, input_feats:list, models:list) -> np.array:
    preds = [
        model.predict(dataframe[input_feats], prediction_type="Probability")[:,1] 
        for model in models
    ]
    return np.mean(preds, axis=0)    

In [ ]:
test = pd.read_parquet("../data/processed/dsv06/test.parquet")
sub = pd.read_csv("../data/raw/sample_submission.csv")

In [ ]:
%%time

all_preds = list()

for repetition in range(N_REPEATS):
    if "prediction" in sub.columns:
        sub.drop("prediction", axis=1, inplace=True)
    if "prediction" in test.columns:
        test.drop("prediction", axis=1, inplace=True)
        
    models = all_models[repetition]
    preds = make_predictions(test, input_feats, models)
    all_preds.append(preds)
       
    test["prediction"] = preds
    sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
    assert sub.prediction.isna().sum() == 0
    sub.to_csv(SUB_PATH/f"submission-cv{repetition}.csv", index=False)

In [ ]:
%%time
# predict using all the trained models
if "prediction" in sub.columns:
    sub.drop("prediction", axis=1, inplace=True)
if "prediction" in test.columns:
    test.drop("prediction", axis=1, inplace=True)

test["prediction"] = np.mean(all_preds, axis=0)
sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
assert sub.prediction.isna().sum() == 0
sub.to_csv(SUB_PATH/f"submission-all.csv", index=False)

***